### LLM = Large language model
#### Langchain은 LLM과 Chat model 두 가지를 모두 지원



In [ ]:
# LLMs and Chat Models
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

llm = OpenAI(model_name="gpt-3.5-turbo-0125")

chat = ChatOpenAI()

a = llm.predict("행성에 대해 알려줘")
b = chat.predict("행성은 뭐가 있어")

a, b

In [4]:
# Predict Messages
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage, SystemMessage
# SystemMessage: 우리가 LLM에 설정들을 제공하기 위한 message

chat = ChatOpenAI(temperature=0.1)
# temperature: 얼마나 창의적인가 0~1

messages = [
    SystemMessage(content="당신은 지리 전문가야, 당신은 한국어로만 답변해야 합니다."),
    AIMessage(content="안녕하세요. 저는 장태훈입니다."),
    HumanMessage(content="한국과 일본 사이의 거리는 어떻게 되나요? 그리고 당신의 이름은 어떻게 되나요")
]

chat.predict_messages(messages)

AIMessage(content='한국과 일본 사이의 거리는 직선거리로 약 900km 정도입니다. 제 이름은 장태훈입니다.')

#### prompt:  LLM과 의사소통할 수 있는 유일한 방법, 성능이 좋으면 답변도 좋다.
-  from langchain.prompts import PromptTemplate, ChatPromptTemplate
> - PromptTemplate: string을 이용해 template을 만든다.
> - ChatPromptTemplate: message로부터 template을 만든다.


In [6]:
# Prompt Templates

from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate

# messages = [
#     SystemMessage(content="당신은 지리 전문가야, 당신은 {language}로만 답변해야 합니다."),
#     AIMessage(content="안녕하세요. 저는 {name}입니다."),
#     HumanMessage(content="{country_a}과 {country_b} 사이의 거리는 어떻게 되나요? 그리고 당신의 이름은 어떻게 되나요")
# ]

chat = ChatOpenAI(temperature=0.1)

template = PromptTemplate.from_template("{country_a}과 {country_b} 사이의 거리는 어떻게 되나요? 그리고 당신의 이름은 어떻게 되나요")
prompt = template.format(country_a="한국", country_b="일본")
print(prompt)

chat.predict(prompt)


'한국과 일본 사이의 거리는 해상 거리로 약 900km 정도이며, 항공편으로는 약 2시간 정도 소요됩니다. \n\n제 이름은 AI 어시스턴트입니다. 저를 통해 궁금한 것이 있으면 언제든지 물어봐주세요!'

In [9]:
# Chat Prompt Template
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

chat = ChatOpenAI(temperature=0.1)

# list 전달
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리 전문가야, 당신은 {language}로만 답변해야 합니다."),
    ("ai", "안녕하세요. 저는 {name}입니다."),
    ("human", "{country_a}과 {country_b} 사이의 거리는 어떻게 되나요? 그리고 당신의 이름은 어떻게 되나요")
])
prompt = template.format_messages(
    language= "한국어",
    name = "김쫀떡",
    country_a = "한국",
    country_b = "일본"
)
print(prompt)

chat.predict_messages(prompt)

[SystemMessage(content='당신은 지리 전문가야, 당신은 한국어로만 답변해야 합니다.'), AIMessage(content='안녕하세요. 저는 김쫀떡입니다.'), HumanMessage(content='한국과 일본 사이의 거리는 어떻게 되나요? 그리고 당신의 이름은 어떻게 되나요')]


AIMessage(content='한국과 일본 사이의 거리는 직선거리로 약 900km 정도입니다. 제 이름은 김쫀떡입니다.')

In [16]:
# OutputParser and LCEL
# LangChain expression language는 코드를 줄여준다. / 다양한 template과 LLM호출, 서로 다른 응답을 할께 사용하게 해준다.
# OutputParser: LLM의 응답을 변해야할 때가 있기 때문에 필요

from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate

from langchain.schema import BaseOutputParser

chat = ChatOpenAI(temperature=0.1)

class CommaOutputParser(BaseOutputParser):
    def parse(self, text):
        items = text.strip().split(",")
        return list(map(str.strip, items))
    
template = ChatPromptTemplate.from_messages([
    ("system", "당신은 list 생성 기계입니다. 입력받은 질문들은 콤마로 구분해서 최대 {max_items}만큼 list로 답해주세요. list가 아닌 것으로 답하지 마세요."),
    ("human", "{question}")
])
prompt = template.format_messages(
    max_items = 10,
    question = "행성이 무엇이 있는지 알려줘"
)

result = chat.predict_messages(prompt)

p = CommaOutputParser()
p.parse(result.content)

['수성', '금성', '지구', '화성', '목성', '토성', '천왕성', '해왕성']

#### Chain
> - chain끼리도 결합할 수 있다.
> - - chain_one = template | chat | CommaOutputParser()
> - - chain_two = template_2 | chat | CommaOutputParser()
> - - all = chain_one | chain_two | CommaOutputParser()

In [17]:
# Chain
chain = template | chat | CommaOutputParser()

chain.invoke({
    "max_items": 5,
    "question": "포켓몬이 모야?"
})

['포켓몬은 포켓몬스터라고 불리는 가상 생명체들을 일컫는 말입니다.']